In [1]:
# safe divide function to stop zero counts from causing problems. This is zero counts for number of words
# which can happen sometimes when texts are small, texts are empty, or content words are removed from small texts
def safe_divide(a, b): #function calls two arguments. Here it will be a POS count and number of words
    if b != 0: #
        return a/b
    else:
        return 0

In [2]:
import os
# Change to the working directory
os.chdir('/Users/zepedalab/Desktop/MATHia_NLP_YB')

#Lab wd:/Users/zepedalab/Desktop/MATHia_NLP_YB
#Home wd: /Users/yuchengbao/Desktop/ALL Lab/MATHia_NLP

import pandas as pd
import re
import spacy
!python3 -m spacy download en_core_web_sm

nlp = spacy.load("en_core_web_sm")

/Users/zepedalab/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


/Users/zepedalab/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.1 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [3]:
#import the dataset
text_df = pd.read_excel('/Users/zepedalab/Desktop/MATHia_NLP_YB/E1C3 Codes (COMPLETE).xlsx')

#home wd: /Users/yuchengbao/PycharmProjects/JupyterProject/models/Practice Dataset.xlsx
#lab wd: /Users/zepedalab/Desktop/MATHia_NLP_YB/E1C3 Codes (COMPLETE).xlsx

In [4]:
# pre-process the text data (text_df)
text_df['input'] = text_df['input'].astype(str) # convert the text input to string type
text_df['input'] = text_df['input'].str.lower() # convert the text to lower case

text_df_3 = text_df[text_df['input'].str.len() >= 3] #can adjust the threshold here
short_input_df = text_df[text_df['input'].str.len() < 4]


In [5]:
short_input_df

,Anon.Student.Id,input,Invalid Response,Self-Efficacy,Utility Value,Intrinsic Values (replaced Interest Enhancement),Opportunity cost \n(outside effort cost),Effort,Extrinsic Rewards/ Self-Consequating,Environmental Control,...,Fear of Failure,Attention,Willpower,Seeking help from others,Emotion Regulation,Did Not Motivate,I Don't Know,No Strategy Listed,Count,Notes
1,stu_GDLLQ68531,no,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,NaN
2,stu_KAIDW62100,nan,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
3,stu_KAIDW62100,nan,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
5,stu_TMJAI99056,nn,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
6,stu_QRVFN34916,e,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,sch_NQNWK22810,nan,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
452,sch_NQNWK22810,idk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,NaN
453,sch_NQNWK22810,nan,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
454,sch_NQNWK22810,nan,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN


In [6]:
#Method 1: detect the gibberish and cross-check with the human labels
#from langdetect import detect_langs

#Method 2: gibberish detector based on training model

from gibberish_detector import detector
#os.chdir('/Users/yuchengbao/Desktop/ALL Lab/MATHia_NLP')
#os.system('gibberish-detector train big.txt > gibberish-detector.model')
Detector = detector.create_from_model('/Users/zepedalab/PycharmProjects/MATHia-NLP-YB/gibberish-detector.model')
#lab wd: /Users/zepedalab/PycharmProjects/MATHia-NLP-YB/gibberish-detector.model

In [7]:
#using detector to initially test the gibberish
# Create an empty dataframe to store gibberish rows
gibberish_df = pd.DataFrame(columns=text_df_3.columns)

# Iterate through rows and separate gibberish rows
for index, row in text_df_3.iterrows():
    try:
        # Check if the row is gibberish
        is_gibberish = Detector.is_gibberish(row['input'])

        # If it is gibberish, add to the gibberish_df and remove from text_df_3
        if is_gibberish:
            gibberish_df = pd.concat([gibberish_df, row.to_frame().T], ignore_index=True)
            #text_df_3 = text_df_3.drop(index)

        print(row['input'], is_gibberish)

    except Exception as e:
        print(f"Error processing row {index}: {e}")


that i will awaly do better then last time False
nan False
nan False
. jjjnnk.nj..mi,.mk.ki.kii..l.kin. True
nan False
focus in classs or ask for help in work from the teacher or ask the fellow classmates. False
i am and i was hecking my work and if it didnt make since i would use the hit to help and solve it. False
i motivated myself by thinking about what would happen if i fail. False
less work to do after False
i worked hard and kept going. False
i worked hard and try to get it done False
idk so i can get out of school False
i just took my time and focus False
i just did it. False
keep tring?.. False
one of the many ways that i motivate myself is by saying that it will be ok and i is just a question. False
i motivated myslef by reminding myself that there is reward for hard work. False
i set my goal to something and do it so in the end i can do what i want False
i took my time and focused and didnt get too stressed False
thinking about going outside False
i kept thinking about the g

In [8]:
gibberish_df

,Anon.Student.Id,input,Invalid Response,Self-Efficacy,Utility Value,Intrinsic Values (replaced Interest Enhancement),Opportunity cost \n(outside effort cost),Effort,Extrinsic Rewards/ Self-Consequating,Environmental Control,...,Fear of Failure,Attention,Willpower,Seeking help from others,Emotion Regulation,Did Not Motivate,I Don't Know,No Strategy Listed,Count,Notes
0,stu_KAIDW62100,". jjjnnk.nj..mi,.mk.ki.kii..l.kin.",1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
1,stu_HMCKN95919,bruhhhh,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
2,sch_NQNWK22810,jjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjjj...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
3,sch_NQNWK22810,3434r 34-gt4ktn'q,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
4,sch_NQNWK22810,idk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,NaN
5,sch_NQNWK22810,yayyyyy,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
6,sch_NQNWK22810,idk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,NaN
7,sch_NQNWK22810,idk,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,NaN
8,sch_HTPUV14121,kjh,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN
9,sch_NQNWK22810,kklj,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,NaN


In [10]:
#Languag Detector - Method 1:
#lingua import: https://github.com/pemistahl/lingua-py
!pip install lingua-language-detector
from lingua import Language, LanguageDetectorBuilder
languages = [Language.ENGLISH,Language.SPANISH]
detector = LanguageDetectorBuilder.from_languages(*languages).build()

# Create an empty dataframe to store non-English rows
non_eng_df = pd.DataFrame(columns=text_df_3.columns)

# Iterate through rows and check for non-English texts (confidence_values = 0)
for index, row in text_df_3[:100].iterrows():
    try:
        # Get the confidence values for the language detection
        confidence_values = detector.compute_language_confidence_values(row['input'])
        # Extract the confidence value for English
        english_confidence = next((cv.value for cv in confidence_values if cv.language == Language.ENGLISH), 0)
        # Check if confidence value for English is 0 (i.e., non-English)
        if english_confidence < 0.8:
            # Add the row to the non_eng_df
            non_eng_df = pd.concat([non_eng_df, row.to_frame().T], ignore_index=True)
            # Remove the row from the original dataframe (if needed)
            #text_df_3 = text_df_3.drop(index)
        #print(row['input'], confidence_values.get(Language.ENGLISH, 0))
    except Exception as e:
        print(f"Error processing row {index}: {e}")

zsh:1: command not found: pip


ModuleNotFoundError: No module named 'lingua'

In [48]:
non_eng_df

,Anon.Student.Id,input,Invalid Response,Self-Efficacy (used to be Efficacy Management/ Efficacy Self-Talk),Utility Value,Intrinsic Values (replaced Interest Enhancement),Opportunity cost (outside effort cost),Effort,Extrinsic Rewards/Self-Consequating,Environmental Control,...,Fear of Failure,Attention,Willpower,Seeking help from others,Emotion Regulation,Did Not Motivate,I Don't Know,No Strategy Listed,Count,Unnamed: 22
0,stu_KAIDW62100,nan,1,0,0,0,0,0,0,0.0,...,0.0,0,0,0,0,0.0,0,0.0,1,NaN
1,stu_KAIDW62100,nan,1,0,0,0,0,0,0,0.0,...,0.0,0,0,0,0,0.0,0,0.0,1,NaN
2,stu_KAIDW62100,". jjjnnk.nj..mi,.mk.ki.kii..l.kin.",1,0,0,0,0,0,0,0.0,...,0.0,0,0,0,0,0.0,0,0.0,1,NaN
3,stu_PLFUT13166,nan,1,0,0,0,0,0,0,0.0,...,0.0,0,0,0,0,0.0,0,0.0,1,NaN
4,stu_ZWFZI88907,i just did it.,0,0,0,0,0,0,0,0.0,...,0.0,0,0,0,0,1.0,0,0.0,1,NaN
5,stu_DJOZT64571,write the problem then solve it,0,0,0,0,0,0,0,0.0,...,0.0,0,0,0,0,0.0,0,1.0,1,NaN
6,stu_XZIES65368,ned,1,0,0,0,0,0,0,0.0,...,0.0,0,0,0,0,0.0,0,0.0,1,NaN
7,stu_EMFNA59963,enter,1,0,0,0,0,0,0,0.0,...,0.0,0,0,0,0,0.0,0,0.0,1,NaN
8,stu_RPMFP80051,nan,1,0,0,0,0,0,0,0.0,...,0.0,0,0,0,0,0.0,0,0.0,1,NaN
9,stu_MWIDT42542,yes,0,0,0,0,0,0,0,0.0,...,0.0,0,0,0,0,0.0,0,1.0,1,NaN


In [11]:
confidence_values = detector.compute_language_confidence_values("holy moo")
print(confidence_values)

[ConfidenceValue(language=Language.ENGLISH, value=0)]


In [12]:
#generate a new column 'gibberish_detected' to store the gibberish detection results
text_df['gibberish_detected'] = 0  # default column

for index, row in text_df.iterrows():
    try:
        langs = detect_langs(row['input'])

        if len(langs) > 0:
            lang = langs[0].lang
            prob = langs[0].prob

            # Mark as gibberish if not English OR English confidence < threshold
            if langs[0].lang != 'en' and langs[0].prob > 0.8:
                text_df.at[index, 'gibberish_detected'] = 1
            else:
                text_df.at[index, 'gibberish_detected'] = 0
    except:
        text_df.at[index, 'gibberish_detected'] = 1  # Mark as gibberish if detection fails

# compare with human labels
text_df['gibberish_match'] = text_df['gibberish_detected'] == text_df['Invalid Response']
# calculate accuracy
accuracy = text_df['gibberish_match'].mean()
print(f'Gibberish detection accuracy: {accuracy:.2%}')

#pull out the Anon.Student.Id, input, Invalid Response, gibberish_detected, gibberish_match columns for review
text_df_quick_review = text_df[['Anon.Student.Id', 'input', 'Invalid Response', 'gibberish_detected', 'gibberish_match']]

Gibberish detection accuracy: 36.56%


In [13]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# True labels (from human-coded data)
y_true = text_df['Invalid Response']
# Predicted labels (from your detection code)
y_pred = text_df['gibberish_detected']

# 1️⃣ Accuracy: How many total predictions were correct
accuracy = accuracy_score(y_true, y_pred)

# 2️⃣ Precision (Of the items flagged as gibberish, how many were actually gibberish), Recall (How many actual gibberish cases were caught), F1
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# 3️⃣ Full classification report
print("=== Classification Report ===")
print(classification_report(y_true, y_pred, target_names=["Valid", "Gibberish"]))

# 4️⃣ Confusion matrix
cm = confusion_matrix(y_true, y_pred)
print("\n=== Confusion Matrix ===")
print(cm)

# 5️⃣ Summary output
print(f"\nAccuracy: {accuracy:.2%}")
print(f"Precision: {precision:.2%}")
print(f"Recall: {recall:.2%}")
print(f"F1-score: {f1:.2%}")

=== Classification Report ===
              precision    recall  f1-score   support

       Valid       0.00      0.00      0.00       295
   Gibberish       0.37      1.00      0.54       170

    accuracy                           0.37       465
   macro avg       0.18      0.50      0.27       465
weighted avg       0.13      0.37      0.20       465


=== Confusion Matrix ===
[[  0 295]
 [  0 170]]

Accuracy: 36.56%
Precision: 36.56%
Recall: 100.00%
F1-score: 53.54%


/Users/yuchengbao/PycharmProjects/JupyterProject/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/yuchengbao/PycharmProjects/JupyterProject/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/yuchengbao/PycharmProjects/JupyterProject/.venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to contr

In [14]:
#output the review file
text_df_quick_review.to_excel('/Users/yuchengbao/PycharmProjects/JupyterProject/models/gibberish_detection_review.xlsx', index=False)

In [15]:
text_data_docs = list(nlp.pipe(text_df.input)) #spacy the texts

In [16]:
# preview the text_df
text_df

,Anon.Student.Id,input,Invalid Response,Self-Efficacy (used to be Efficacy Management/ Efficacy Self-Talk),Utility Value,Intrinsic Values (replaced Interest Enhancement),Opportunity cost (outside effort cost),Effort,Extrinsic Rewards/Self-Consequating,Environmental Control,...,Willpower,Seeking help from others,Emotion Regulation,Did Not Motivate,I Don't Know,No Strategy Listed,Count,Unnamed: 22,gibberish_detected,gibberish_match
0,stu_NWRXR49435,that i will awaly do better then last time,0,0,0,0,0,0,0,0.0,...,0,0,0,0.0,0,0.0,1,NaN,1,False
1,stu_GDLLQ68531,no,0,0,0,0,0,0,0,0.0,...,0,0,0,1.0,0,0.0,1,NaN,1,False
2,stu_KAIDW62100,nan,1,0,0,0,0,0,0,0.0,...,0,0,0,0.0,0,0.0,1,NaN,1,True
3,stu_KAIDW62100,nan,1,0,0,0,0,0,0,0.0,...,0,0,0,0.0,0,0.0,1,NaN,1,True
4,stu_KAIDW62100,". jjjnnk.nj..mi,.mk.ki.kii..l.kin.",1,0,0,0,0,0,0,0.0,...,0,0,0,0.0,0,0.0,1,NaN,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460,sch_NQNWK22810,i dont know i jjust worked hard,0,0,0,0,0,0,0,0.0,...,0,0,0,0.0,1,0.0,1,NaN,1,False
461,sch_HTPUV14121,i didnt,0,0,0,0,0,0,0,0.0,...,0,0,0,1.0,0,0.0,1,NaN,1,False
462,sch_NQNWK22810,i dont want to fail so i kept telling myself t...,0,0,0,0,0,0,0,0.0,...,0,0,0,0.0,0,0.0,1,NaN,1,False
463,sch_HTPUV14121,",ndcfk,sdj",1,0,0,0,0,0,0,0.0,...,0,0,0,0.0,0,0.0,1,NaN,1,True
